# <center> Lecture15 : Hierarchical Models (2)</center>  
 
## <center> Instructor: Dr. Hu Chuan-Peng </center> 

## bambi code  

【新增补充内容放到这个下面：复制即可】

### 补充内容：分层模型中的组层面预测因子(group-level predictors)

🤔 在上面的例子中，我们考虑了个体压力对于自我控制的影响，也考虑了不同站点对于自我控制的影响。然而，我们是否考虑过不同站点的其他特性对于自我控制的影响呢？ 
* 例如，不同站点对应了不同国家和地区，这些地区的经济发展水平、教育水平、人口结构等特性是否会对自我控制产生影响呢？




**例如，我们假设：地区(站点)经济水平越发达，个体自我控制能力越强**

组层面预测因子(group-level predictors)是指在组间(站点)层面的特征，这是相对于组内预测因子(group-level predictors)而言的。
* 例如，个体压力分数(stress)是组内预测因子。这个值在不同个体间具有不同值。
* 而地区的经济发展水平(GDP)是组层面预测因子。这些值在不同站点(site)间不同，但是在站点内的个体间保持相同。

![Image Name](https://cdn.kesci.com/upload/s5v47lfq02.png?imageView2/0/w/750/h/750)


【下面给个展示数据的代码；然后展示一个回归图(需要在站点上求平均)，给出 GDP越高，自我控制越高的趋势】

**构建具有组层面预测因子的分层模型**

回顾之前的仅包含变化的截距的分层模型：

$$  
\begin{array}{rll}  
Y_{ij} | \beta_{0j}, \beta_1, \sigma_y & \sim N(\mu_{ij}, \sigma_y^2) \;\; \text{ with } \;\;  \mu_{ij} = \beta_{0j} + \beta_1 X_{ij} & \text{(Layer1: 每个站点内的线性模型)} \\  
\beta_{0j} | \beta_0, \sigma_0  & \stackrel{ind}{\sim} N(\beta_0, \sigma_0^2) & \text{(Layer2: 截距在站点间的变化)} \\  
\beta_{0}  & \sim N(0, 50^2) & \text{(Layer3: 全局参数的先验)} \\  
\beta_1  & \sim N(0, 5^2) & \\  
\sigma_y & \sim \text{Exp}(1)    & \\  
\sigma_0 & \sim \text{Exp}(1).    & \\  
\end{array}  
$$

🤔 如果我们想考虑组层面预测因子GDP，那么该变量该放在哪个层呐(Layer)？
* 由于组层面预测因子是组间(站点$j$)层面的特征，因此该变量主要影响截距在站点间的变化。
* 公式写作： $\beta_{0j} = \gamma_0 + \gamma_1 U_j$
  * 其中，$U_j$是站点$j$的组层面预测因子
  * $\gamma_1$是组层面预测因子$U_j$的回归系数；而$\gamma_0$为站点间的变化的截距。
  * 因此，每个站点的截距 $\beta_{0j}$，受到组层面的预测因子$U_j$的影响。
* 也可以写作 $\mu_{ij} = \beta_{0j} + \beta_1 X_{ij} = (\gamma_0 + \gamma_1 U_j) + \beta_1 X_{ij}$

**总的公式** 

将 $\beta_{0j} = \gamma_0 + \gamma1 U_j$ 放入之前的公式，并且为新参数$\gamma_0$和$\gamma_1$引入先验分布：

$$  
\begin{array}{rll}  
Y_{ij} | \beta_{0j}, \beta_1, \sigma_y & \sim N(\mu_{ij}, \sigma_y^2) \;\; \text{ with } \;\;  \mu_{ij} = \beta_{0j} + \beta_1 X_{ij} & \text{(Layer1: 每个站点内的线性模型)} \\  
\beta_{0j} | \gamma_0, \gamma_1, \sigma_0  & \stackrel{ind}{\sim} N(\gamma_0 + \gamma_1 U_j, \sigma_0^2) & \text{(Layer2: 截距在站点间的变化)} \\  
\gamma_0  & \sim N(0, 50^2) & \text{(Layer3: 全局参数的先验)} \\  
\gamma_1 & \sim N(0, 5^2) & \\  
\beta_1  & \sim N(0, 5^2) & \\  
\sigma_y & \sim \text{Exp}(1)    & \\  
\sigma_0 & \sim \text{Exp}(1).    & \\  
\end{array}  
$$

【pymc 公式】
【模型图】
【两个gamma参数的后验】
【gamma形成的回归预测图】

当模型变得复杂， 在 PyMC 中定义分层模型将变得困难。

使用 Bambi 可以轻松定义以上模型，其模型定义公式为 `"scontrol ~ 1 + GDP + stress + (1|Site)"`
* 可以发现，在 Bambi 中，组层面的预测因子(GDP)的定义方式和个体层面的预测因子(stress)的定义方式一致。
* 也就是说，Bambi 支持用同一种公式定义个体和组层面的预测因子。

【bambi 公式】
【模型图】

## Hierarchical logistic regression

> 在之前的课程中，除了正态回归模型，我们还介绍过logistic回归模型、泊松回归模型和负二项回归模型。这些模型同样可以和层级模型结合  

* 在lec13中我们使用回避依恋分数来预测个体的恋爱情况，假设这一线性关系在不同文化中有不同的表现，我们也可以把站点信息考虑在内  

* 对于因变量为离散变量的情况，我们需要使用广义线性模型(Generalized linear model，GLM)。  

* 其特点为：  
	- 分布簇 (dist)不再局限于正态分布，而是允许其他不同的分布，比如 $y \sim Bernoulli(p)$  
	- 需要 **链接函数$g()$** 将 $\alpha + \beta * x$  映射到 $p$所在的范围  

| 一般线性模型 | 广义线性模型 |  
|---|---|  
| $y \sim Normal(\mu, \sigma)$ | $y \sim dist(p)$ |  
| $\mu = \alpha + \beta *x$ | $p = g(z)$|  
|  | $z = \alpha + \beta *x$|  


### 层级模型  

$$  

\begin{array}{rll}  
Y_{ij}|\beta_{0j},\beta_{1j} & \sim \text{Bern}(\pi_{ij})\; \text{ with } \;\; \pi_i = \frac{e^{\beta_{0j} + \beta_{1j} X_{ij}}}{1 + e^{\beta_{0j} + \beta_{1j} X_{ij}}} \\  
&& \text{(每个站点 $j$内的线性模型)}\\  
\beta_{0j} | \beta_0, \sigma_0    & \stackrel{ind}{\sim} N(\beta_0, \sigma_0^2) & \text{(截距在站点间的变化)} \\  
\beta_{1j} | \beta_1, \sigma_1    & \stackrel{ind}{\sim} N(\beta_1, \sigma_1^2) & \text{(斜率在站点间的变化)} \\  
\beta_{0}  &  \sim N\left(0, 0.5^2 \right) & \text{(全局参数的先验)}\\  
\beta_1  &  \sim N\left(0, 0.5^2 \right) & \\  
\sigma_0 & \sim \text{Exp}(1). & \\  
\sigma_1 & \sim \text{Exp}(1). & \\  
\end{array}  

$$